# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [69]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [70]:
document = ET.parse('/Users/michaelcaruana/Desktop/springboard_dsi/data_wrangling_xml/data/mondial_database.xml')

### 1. 10 countries with the lowest infant mortality rates

In [71]:
infant_mortality = []
country_list = []
for element in document.getroot():
    for subelement in element.getiterator('infant_mortality'):
        country_list.append( element.find('name').text )
        infant_mortality.append(np.float(subelement.text))
data = {'Country':country_list, 'Infant_mortality':infant_mortality}
inf_mort = pd.DataFrame(data=data)
inf_mort.sort_values('Infant_mortality',ascending=True).head(10)

,Country,Infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 2. 10 cities with the largest population

In [72]:
city = []
year = []
population = []

for element in document.getroot():
    for subelement in element.getiterator('city'):
        for subsubelement in subelement.getiterator('population'):
            if (subsubelement.get('measured')=='census'):
                city.append(subelement.find('name').text)
                year.append(np.float(subsubelement.get('year')))
                population.append(np.float(subsubelement.text))

data = {'City':city,'Population':population,'Year':year}
dataframe = pd.DataFrame(data=data).sort('Population',ascending=False)
dataframe.head(10)

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,City,Population,Year
2190,Shanghai,22315474,2010
2189,Shanghai,15758892,2000
2669,Delhi,12877470,2001
2599,Mumbai,12442373,2011
2598,Mumbai,11914398,2001
2187,Beijing,11716620,2010
979,Moskva,11612885,2010
5605,São Paulo,11152344,2010
2193,Tianjin,11090314,2010
2072,Guangzhou,11071424,2010


### 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [73]:
country = []
ethnic_group = []
population = []
ethnic_population = []

for element in document.getroot():
    for subelement in element.getiterator('country'):
        for i in range(0,len(element.findall('ethnicgroup'))):
            country.append(element.find('name').text)
            ethnic_group.append(element.findall('ethnicgroup')[i].text)
            population.append(np.float(element.findall('population')[-1].text))
            ethnic_population.append(np.float(element.findall('population')[-1].text)*np.float(element.findall('ethnicgroup')[i].attrib['percentage']))

data = {'Country':country,
        'Ethnic_Group':ethnic_group,
        'Overall_Population':population,
        'Ethnic_Population':ethnic_population}

population_table = pd.DataFrame(data=data)
total_ethnic_population = population_table.groupby('Ethnic_Group')['Ethnic_Population'].sum().reset_index().sort('Ethnic_Population',ascending=False)
total_ethnic_population.head(10)

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Ethnic_Group,Ethnic_Population
113,Han Chinese,1.245059e+11
120,Indo-Aryan,8.718156e+10
89,European,4.948722e+10
2,African,3.183251e+10
77,Dravidian,3.027137e+10
176,Mestizo,1.577344e+10
42,Bengali,1.467769e+10
217,Russian,1.318570e+10
128,Japanese,1.265342e+10
163,Malay,1.219936e+10


### 4.a Name and country of longest river 

In [74]:
#Realized that I need to create mapping of country code to country name
country_map = {}
for element in document.getiterator('country'):
     country_map.update({element.attrib['car_code']:element.find('name').text})

In [75]:
river = []
length = []
country_code = []
for element in document.getroot():
    for subelement in element.getiterator('river'):
        if (subelement.find('source') is not None) & (element.find('length') is not None):
            river.append(element.find('name').text)
            length.append(np.float(element.find('length').text))
            country_code.append(element.find('source').attrib['country'].split()[0])

data = {'country_code':country_code,
        'length':length,
        'name':river}

rivers = pd.DataFrame(data=data)
rivers['country'] = rivers['country_code'].apply(lambda x: country_map[x])
rivers = rivers.sort('length',ascending=False)
rivers.head(10)

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,country_code,length,name,country
174,PE,6448,Amazonas,Peru
137,CN,6380,Jangtse,China
136,CN,4845,Hwangho,China
123,R,4400,Lena,Russia
201,ZRE,4374,Zaire,Zaire
138,CN,4350,Mekong,China
115,KAZ,4248,Irtysch,Kazakhstan
186,RG,4184,Niger,Guinea
160,USA,4130,Missouri,United States
119,R,4092,Jenissej,Russia


### 4.b Name and country of largest lake

In [76]:
lake = []
size = []
country_code = []

for element in document.getiterator('lake'):
    if (element.attrib['country'] is not None) & (element.find('area') is not None):
        lake.append(element.find('name').text)
        size.append(np.float(element.find('area').text))
        country_code.append(element.attrib['country'].split()[0])

data = {'country_code':country_code,
        'size':size,
        'name':lake}

lake = pd.DataFrame(data=data)
lake['country'] = lake['country_code'].apply(lambda x: country_map[x])
lake = lake.sort('size',ascending=False)
lake.head(10)

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,country_code,name,size,country
54,R,Caspian Sea,386400,Russia
107,CDN,Lake Superior,82103,Canada
79,EAT,Lake Victoria,68870,Tanzania
104,CDN,Lake Huron,59600,Canada
106,USA,Lake Michigan,57800,United States
47,IL,Dead Sea,41650,Israel
81,ZRE,Lake Tanganjika,32893,Zaire
96,CDN,Great Bear Lake,31792,Canada
43,R,Ozero Baikal,31492,Russia
87,MW,Lake Malawi,29600,Malawi


### 4.c Name and country of airport at highest elevation

In [77]:
airport = []
elevation = []
country_code = []
for element in document.getroot():
    for subelement in element.getiterator('airport'):
        for subsubelement in subelement.getiterator('elevation'):
            airport.append(subelement.find('name').text)
            elevation.append(subsubelement.text )
            country_code.append(subelement.attrib['country'])
            
data = {'airport':airport,
        'elevation':elevation,
        'country_code':country_code}

airport_elevation = pd.DataFrame(data=data)
airport_elevation['elevation'] = airport_elevation['elevation'].astype(float)
airport_elevation['country'] = airport_elevation['country_code'].apply(lambda x: country_map[x])
airport_elevation = airport_elevation.sort('elevation',ascending=False)
airport_elevation.head(10)

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,airport,country_code,elevation,country
80,El Alto Intl,BOL,4063,Bolivia
219,Lhasa-Gonggar,CN,4005,China
241,Yushu Batang,CN,3963,China
813,Juliaca,PE,3827,Peru
815,Teniente Alejandro Velasco Astete Intl,PE,3311,Peru
82,Juana Azurduy De Padilla,BOL,2905,Bolivia
334,Mariscal Sucre Intl,EC,2813,Ecuador
805,Coronel Fap Alfredo Mendivil Duarte,PE,2719,Peru
807,Mayor General FAP Armando Revoredo Iglesias Ai...,PE,2677,Peru
692,Licenciado Adolfo Lopez Mateos Intl,MEX,2581,Mexico
